# Bitbubble

Simplified ABM model that links the **"How to grow a bubble: A model of myopic adapting agents"** with the **"Leader–follower model for agent based simulation of social collective behavior during egress"** in the bitcoin bubble model space

In [271]:
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib as mpl
from mesa import Agent, Model
import matplotlib.pyplot as plt
from mesa.time import RandomActivation
%matplotlib inline
#print(mpl.style.available)
mpl.style.use('ggplot')
plt.xkcd()
np.random.seed(seed=42)

In [390]:
class Investor(Agent):
    """An investor with fixed initial cash and coins."""
    def __init__(self, unique_id, model, C1=1, C2=1, C3=1
                 , alpha=0.95, Omega=2, g=0.02, cash=1.0, coins=1.0):
        super().__init__(unique_id, model)
        self.cash = cash
        self.coins = coins
        self.alpha = alpha
        self.Omega = Omega
        self.g = g
        #TODO: how to generate closed interval random numbers?
        self.c1 = np.random.uniform(0.0, C1)
        self.c2 = np.random.uniform(0.0, C1)
        self.c3 = np.random.uniform(0.0, C3)
        self.w_min = np.random.uniform(0.0, Omega)
        self.opinions = []
        self.actions = []
        self.private_info = [self.collect_private_info()]
        
    def step(self):
        time = self.model.t
        E = self.collect_private_info()
        self.private_info.append(E)
        opinion = self.get_market_opinion(E)
        action, volume = self.get_actions(opinion, time)
        #print("Agent", self.unique_id, "w_min", self.w_min, opinion, action, volume)
        self.model.add_order(self.unique_id, action, volume)
        
    def collect_private_info(self):
        return np.random.normal()
    
    def get_market_opinion(self, E):
        t = self.model.t
        opinion = (self.c1) * sum( [(self.get_neighbor_factor(neighbor, t-1) 
                           * self.get_neighbor_expected_action(neighbor, t)) 
                          for neighbor in self.get_neighbors()] ) 
        + (self.c2 * self.get_news_factor(t-1) * self.model.newspaper(t)) 
        + (self.c3 * E)
        return opinion
        
    def get_neighbors(self):
        return [t[1] for t in self.model.G.edges(self.unique_id)]
        
    def get_neighbor_factor(self, neighbor_id, time):
        return np.random.uniform(-1,1)
    
    def get_neighbor_expected_action(self, neighbor_id, time):
        return np.random.uniform(-1,1)
    
    def get_news_factor(self, time):
        return np.random.uniform(-1,1)
    
    def get_actions(self, opinion, time):
        s = 0
        v = 0
        if opinion >= self.w_min:
            s = 1
            v = self.g * self.cash / self.model.get_price(time-1)
        elif opinion <= self.w_min * (-1):
            s = -1
            v = self.g * self.coins
        return s, v
        

In [394]:
class MarketMaker(Model):
    """A model with some number of investors."""
    def __init__(self, N, market_depth, k=4):
        self.schedule = RandomActivation(self)
        self.N = N
        self.market_depth = market_depth
        self.t = 0
        self.current_price = 1
        self.list_price = [self.current_price]
        self.list_return = [0]
        self.list_news = []
        self.current_orders = []
        self.G = self.create_network(k)
        # Create agents
        for i in self.G.nodes():
            a = Investor(i, self)
            self.G.add_node(i)
            self.schedule.add(a)
            
    def step(self):
        self.t = self.t + 1
        self.current_orders = list([])
        '''Advance the model by one step.'''
        self.schedule.step()
        r = this.get_return()
        self.list_return.append(r)
        self.current_price = self.calc_price(r)
        self.list_price.append(self.current_price)
        """pending:
            update cash and coins on agents (how?)
            add end conditions (basic, upper limit, lower limit)
        """
        print("orders for t", self.t, self.dict_orders)
        
    def create_network(self, num_edges):
        '''Creates a Barabasi-Albert network with the number of agents and edges.'''
        return nx.generators.random_graphs.barabasi_albert_graph(self.N, num_edges)
    
    def newspaper(self, t):
        available_news = len(self.list_news)
        if available_news <= t:
            for i in range(0, t - (available_news - 1)):
                self.list_news.append(np.random.normal())
        return self.list_news[t]
    
    def get_price(self, t):
        price = self.list_price[0]
        if t > 0 and t < len(self.list_price):
            price = self.list_price[t]
        return price
    
    def add_order(self, unique_id, s=0, v=0.0):
        self.current_orders.append(s*v)
        
    def get_return(self):
        total = sum(self.current_orders)
        return total / (self.market_depth * self.N)
    
    def calc_price(self, r):
        price = np.exp(np.log(self.current_price) + r)
        return price
    

bitmarket = MarketMaker(5, 0.25)
nx.draw_networkx(bitmarket.G)

In [389]:
bitmarket = MarketMaker(5, 0.25)
for i in range(10):
    bitmarket.step()

orders for t 1 {3: 0, 0: 0, 2: 0, 1: 0, 4: 0.02}
orders for t 2 {1: 0, 0: 0, 2: 0, 4: 0, 3: 0}
orders for t 3 {1: 0, 3: 0.02, 4: 0, 2: 0, 0: 0}
orders for t 4 {0: 0, 1: 0, 4: 0, 2: 0, 3: 0}
orders for t 5 {2: 0, 1: 0, 4: 0.02, 3: 0, 0: 0}
-0.6089374044779683 -1 0.02 -0.02
orders for t 6 {1: 0, 3: -0.02, 0: 0, 4: 0, 2: 0}
orders for t 7 {0: 0, 3: 0.02, 2: 0, 4: 0, 1: 0}
-0.4765593007483217 -1 0.02 -0.02
orders for t 8 {1: 0, 3: -0.02, 0: 0, 2: 0, 4: 0}
orders for t 9 {4: 0, 1: 0, 3: 0, 0: 0, 2: 0}
orders for t 10 {2: 0, 0: 0, 4: 0, 1: 0, 3: 0}
